In [6]:
import spacy
import json
from spacy import displacy
from spacy.tokens import DocBin
from tqdm import tqdm
from spacy.util import filter_spans

In [2]:
with open ('../../../raw_data/ra_analyze_annotations(2).json', 'r', encoding='utf-8') as f:
    data = json.load(f)

In [3]:
nlp = spacy.blank('ru')


In [4]:
training_data = []
for example in data['examples']:
    temp_dict = {}
    temp_dict['text'] = example['content']
    temp_dict['entities'] = []
    for annotation in example['annotations']:
        start = annotation['start']
        end = annotation['end']
        label = 'KEYWORDS'
        temp_dict['entities'].append((start, end, label))
    training_data.append(temp_dict)

In [5]:
train = []
for i in range(len(training_data)):
    if len(training_data[i]['entities']) > 0:
        train.append(training_data[i])

In [7]:
doc_bin = DocBin()

In [8]:
for training_example in tqdm(train):
     text = training_example['text']
     labels = training_example['entities']
     doc = nlp.make_doc(text)
     ents = []
     for start, end, label in labels:
         span = doc.char_span(start, end, label=label, alignment_mode="contract")
         if span is None:
             print("Skipping entity")
         else:
             ents.append(span)
     filtered_ents = filter_spans(ents)
     doc.ents = filtered_ents
     doc_bin.add(doc)

doc_bin.to_disk("train_ner.spacy")

  0%|          | 0/106 [00:00<?, ?it/s]

100%|██████████| 106/106 [00:01<00:00, 67.34it/s]


In [10]:
! python -m spacy train config.cfg --output ./ner_model --paths.train train_ner.spacy --paths.dev train_ner.spacy

ℹ Saving to output directory: ner_model
ℹ Using CPU

=========================== Initializing pipeline ===========================
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00    745.33    0.00    0.00    0.00    0.00
  1     200       3351.49  12818.39   15.40   23.03   11.57    0.15
  3     400        491.95   1952.65   38.22   47.65   31.90    0.38
  5     600        101.66   1552.51   54.68   43.57   73.39    0.55
  7     800        116.12   1262.14   71.27   67.20   75.87    0.71
  9    1000        186.05   1120.62   72.28   69.57   75.21    0.72
 11    1200        338.85    911.30   81.65   85.25   78.35    0.82
 13    1400        400.81    830.91   84.26   86.31   82.31    0.84
 15    1600        286.47   